<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-02 13:36:56
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.35 C
-------------------
Today PnL: -1.48 L (-1.08%)
Current PnL: -34.50 L (-22.0%)
CY Booked + Current PnL: -19.76 L (-12.6%)
-------------------
Total profit:  74.80 K
Total loss:  -35.25 L
-------------------
Total Booked + Current PnL: 7.22 L (5.56%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.92%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 1.00 C (74.09%)
Deployed:  1.30 C
Current:  1.35 C
CAGR/XIRR %: 2.04%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,484.83,-5.75,2.15,16.71,19.23,25683.0,3238.0,153698.0,19.88,48.0,L-LC,12.42,182.0,0.13,1.14,23.89,XY25,ATH,MINING
77,TTKPRESTIG,770.00,-2.41,-26.68,36.51,0.09,26978.0,-26886.0,73891.0,71.40,33.0,M-SC,6.51,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES
50,MASFIN,398.61,-5.73,-7.36,31.75,22.05,28818.0,-7215.0,90765.0,-20.05,43.0,H-SC,3.40,168.0,-0.25,0.67,31.93,XR,ATH,FINANCE
33,HCLTECH,1908.19,-2.47,8.96,13.65,23.83,35982.0,21679.0,263603.0,13.61,52.0,X-LC,7.31,13.0,0.60,1.95,26.47,X40,ATH,IT
51,MEDANTA,1486.00,7.98,-6.77,33.08,24.08,39153.0,-8590.0,118360.0,-14.31,48.0,X-SC,9.40,91.0,-0.22,0.88,8.41,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,MEDANTA,1486.0,7.98,-6.77,33.08,24.08,39153.0,-8590.0,118360.0,-14.31,48.0,X-SC,9.40,91.0,-0.22,0.88,8.41,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.0,7.35,-26.03,96.81,45.57,141158.0,-51323.0,145809.0,3148.65,52.0,M-SC,8.27,245.0,-0.36,1.08,10.34,XY24,NTT,AUTO
15,CAMPUS,393.0,5.90,-11.09,49.91,33.28,74447.0,-18612.0,149163.0,-28.38,56.0,M-SC,5.72,221.0,-0.25,1.11,15.61,XY24,NTT,FOOTWEAR
70,SYMPHONY,1306.0,5.69,-28.74,40.28,-0.03,49126.0,-49180.0,121961.0,-42.85,63.0,M-SC,12.88,206.0,-1.00,0.90,12.47,OX40N,NTT,DURABLES
68,STARHEALTH,761.0,4.99,-8.47,60.89,47.26,156677.0,-23811.0,257312.0,24.79,65.0,H-SC,3.31,174.0,-0.15,1.91,37.86,XY24,NTT,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,3033.00,-8.07,-4.34,29.84,24.20,87129.0,-13262.0,291988.0,36.23,40.0,X-SC,2.45,99.0,-0.15,2.16,22.29,X40N,NTT,FINANCE
0,5PAISA,593.00,-6.82,-37.99,82.07,12.90,111197.0,-83005.0,135491.0,99.52,40.0,H-SC,6.24,173.0,-0.75,1.00,12.66,OX40N,NTT,FINANCE
29,GREENPANEL,537.00,-5.95,-42.57,149.22,43.14,156903.0,-77929.0,105149.0,135.79,44.0,M-SC,17.38,231.0,-0.50,0.78,2.83,XY24,NTT,MISC
18,COALINDIA,484.83,-5.75,2.15,16.71,19.23,25683.0,3238.0,153698.0,19.88,48.0,L-LC,12.42,182.0,0.13,1.14,23.89,XY25,ATH,MINING
50,MASFIN,398.61,-5.73,-7.36,31.75,22.05,28818.0,-7215.0,90765.0,-20.05,43.0,H-SC,3.40,168.0,-0.25,0.67,31.93,XR,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,2.07,-0.82,73.69,72.27,123842.0,-1390.0,168058.0,-14.7,46.0,M-LC,2.57,101.0,-0.01,1.25,8.02,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,-0.27,-10.21,131.56,107.91,174401.0,-15080.0,132564.0,-27.04,37.0,M-SC,10.84,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
48,KANSAINER,340.00,4.29,-21.54,44.62,13.47,94411.0,-58077.0,211590.0,-67.73,55.0,H-SC,5.29,159.0,-0.62,1.57,7.59,XY24,NTT,PAINTS
72,TATAELXSI,9161.00,1.12,-22.42,70.18,32.03,71778.0,-29558.0,102277.0,-16.04,49.0,H-SC,6.52,157.0,-0.41,0.76,14.56,OX40N,NTT,IT
66,SIS,528.00,2.74,-22.96,57.40,21.26,49100.0,-25492.0,85540.0,2019.69,55.0,H-SC,4.81,163.0,-0.52,0.63,15.57,OX40N,NTT,MISC
77,TTKPRESTIG,770.00,-2.41,-26.68,36.51,0.09,26978.0,-26886.0,73891.0,71.40,33.0,M-SC,6.51,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,484.83,-5.75,2.15,16.71,19.23,25683.0,3238.0,153698.0,19.88,48.0,L-LC,12.42,182.0,0.13,1.14,23.89,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,2.07,-0.82,73.69,72.27,123842.0,-1390.0,168058.0,-14.70,46.0,M-LC,2.57,101.0,-0.01,1.25,8.02,XR,NTT,IT
1,ABB,7934.00,-1.46,-1.03,47.09,45.57,121922.0,-2707.0,258912.0,-36.40,64.0,H-MC,6.90,121.0,-0.02,1.92,14.97,AR,NTT,ELECTRICAL
39,INDIAMART,4810.62,0.99,-5.59,118.97,106.72,138524.0,-6900.0,116436.0,-53.30,49.0,H-SC,9.05,138.0,-0.05,0.86,16.48,AR,ATH,MISC
26,FINCABLES,1641.55,-0.27,-10.21,131.56,107.91,174401.0,-15080.0,132564.0,-27.04,37.0,M-SC,10.84,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-2.23,-8.17,47.69,35.63,91783.0,-17116.0,192458.0,-21.17,37.0,H-MC,5.49,119.0,-0.19,1.43,7.14,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JIOFIN,387.00,-5.21,-22.40,60.41,24.48,125336.0,-59890.0,207475.0,-23.68,18.0,X-LC,7.63,37.0,-0.48,1.54,20.26,XY24,BTT,FINANCE
12,BATAINDIA,2096.00,-0.67,-45.16,146.53,35.20,103402.0,-58103.0,70567.0,-7.44,19.0,X-SC,19.90,93.0,-0.56,0.52,0.28,X40,NTT,FOOTWEAR
44,ITC,452.00,-2.93,-23.36,44.55,10.78,82052.0,-56138.0,184180.0,-54.23,20.0,X-LC,18.21,1.0,-0.68,1.36,0.00,X40,NTT,FMCG
13,BERGEPAINT,680.00,0.30,-17.29,46.46,21.14,87364.0,-39297.0,188042.0,-23.12,21.0,X-MC,7.28,75.0,-0.45,1.39,0.30,XY24,NTT,PAINTS
6,ASIANPAINT,3460.25,-1.21,-16.15,44.24,20.95,93396.0,-40656.0,211112.0,-16.94,23.0,X-LC,10.58,36.0,-0.44,1.56,13.73,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.58,1.26,-13.02,36.30,18.55,119417.0,-49264.0,328973.0,-25.24,50.0,X-LC,0.56,3.0,-0.41,2.44,11.92,X40,ATH,IT
81,VBL,671.64,-0.11,-5.76,43.96,35.67,130851.0,-18183.0,297659.0,-17.02,38.0,X-LC,1.83,5.0,-0.14,2.21,7.30,X40N,ATH,FMCG
34,HINDUNILVR,2922.00,-1.07,-8.68,24.47,13.67,57443.0,-22313.0,234750.0,-15.44,46.0,X-LC,2.22,9.0,-0.39,1.74,10.82,XY25,NTT,FMCG
5,ANGELONE,3033.00,-8.07,-4.34,29.84,24.20,87129.0,-13262.0,291988.0,36.23,40.0,X-SC,2.45,99.0,-0.15,2.16,22.29,X40N,NTT,FINANCE
36,ICICIGI,2252.93,-0.42,-3.70,23.92,19.34,46096.0,-7401.0,192708.0,-24.04,34.0,X-MC,2.75,71.0,-0.16,1.43,11.65,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-3.43,-31.88,140.96,64.15,228496.0,-75851.0,162100.0,-60.24,30.0,X-SC,7.26,84.0,-0.33,1.20,0.00,XY24,BTT,CEMENT
10,BAJAJHLDNG,14451.52,-2.01,-6.02,37.49,29.22,66990.0,-11438.0,178687.0,-8.67,36.0,X-LC,13.85,29.0,-0.17,1.32,0.00,X40,ATH,FINANCE
44,ITC,452.00,-2.93,-23.36,44.55,10.78,82052.0,-56138.0,184180.0,-54.23,20.0,X-LC,18.21,1.0,-0.68,1.36,0.00,X40,NTT,FMCG
32,HAVELLS,2062.85,-2.33,-19.65,64.37,32.08,169647.0,-64441.0,263550.0,-23.70,25.0,X-MC,7.88,69.0,-0.38,1.95,0.00,X40,ATH,ELECTRICAL
9,BAJAJHFL,181.50,-3.01,-24.53,106.02,55.47,194653.0,-59686.0,183600.0,-33.19,35.0,X-MC,17.13,63.0,-0.31,1.36,0.09,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,0.58,-30.49,105.51,42.84,47674.0,-19822.0,45184.0,-54.27,49.0,X-SC,21.56,83.0,-0.42,0.33,6.21,XY24,NTT,MISC
12,BATAINDIA,2096.00,-0.67,-45.16,146.53,35.20,103402.0,-58103.0,70567.0,-7.44,19.0,X-SC,19.90,93.0,-0.56,0.52,0.28,X40,NTT,FOOTWEAR
58,RELAXO,1176.00,-4.74,-51.74,215.58,52.31,151031.0,-75102.0,70058.0,-48.60,43.0,X-SC,11.49,92.0,-0.50,0.52,4.18,X40N,NTT,FOOTWEAR
51,MEDANTA,1486.00,7.98,-6.77,33.08,24.08,39153.0,-8590.0,118360.0,-14.31,48.0,X-SC,9.40,91.0,-0.22,0.88,8.41,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,1.29,-19.67,78.41,43.32,102592.0,-32032.0,130840.0,-31.02,47.0,X-SC,9.00,90.0,-0.31,0.97,6.24,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,452.00,-2.93,-23.36,44.55,10.78,82052.0,-56138.0,184180.0,-54.23,20.0,X-LC,18.21,1.0,-0.68,1.36,0.00,X40,NTT,FMCG
75,TMPV,600.00,2.53,-26.44,67.18,22.98,111634.0,-59727.0,166171.0,-24.30,56.0,X-LC,4.50,2.0,-0.54,1.23,6.23,XY24,NTT,AUTO
73,TCS,4311.58,1.26,-13.02,36.30,18.55,119417.0,-49264.0,328973.0,-25.24,50.0,X-LC,0.56,3.0,-0.41,2.44,11.92,X40,ATH,IT
81,VBL,671.64,-0.11,-5.76,43.96,35.67,130851.0,-18183.0,297659.0,-17.02,38.0,X-LC,1.83,5.0,-0.14,2.21,7.30,X40N,ATH,FMCG
42,INFY,1972.00,-1.16,10.42,21.58,34.24,74556.0,32598.0,345486.0,-13.85,46.0,X-LC,7.16,6.0,0.44,2.56,19.56,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,-1.18,-34.46,102.76,32.89,51999.0,-26604.0,50602.0,-725.26,48.0,L-MC,11.25,259.0,-0.51,0.37,39.35,XR,NTT,BANKS
14,BSOFT,831.70,2.00,-24.20,98.92,50.79,105465.0,-34037.0,106616.0,-2.59,50.0,H-SC,5.91,171.0,-0.32,0.79,24.14,XR,ATH,IT
18,COALINDIA,484.83,-5.75,2.15,16.71,19.23,25683.0,3238.0,153698.0,19.88,48.0,L-LC,12.42,182.0,0.13,1.14,23.89,XY25,ATH,MINING
68,STARHEALTH,761.00,4.99,-8.47,60.89,47.26,156677.0,-23811.0,257312.0,24.79,65.0,H-SC,3.31,174.0,-0.15,1.91,37.86,XY24,NTT,INSURANCE
33,HCLTECH,1908.19,-2.47,8.96,13.65,23.83,35982.0,21679.0,263603.0,13.61,52.0,X-LC,7.31,13.0,0.60,1.95,26.47,X40,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,GILLETTE,11206.78,-1.51,2.40,29.31,32.42,76205.0,6099.0,259995.0,-15.80,63.0,X-SC,12.67,87.0,0.08,1.93,17.86,X40,ATH,FMCG
62,SATIN,274.00,-1.24,-15.58,79.85,51.83,186249.0,-43036.0,233248.0,-23.78,56.0,H-SC,3.14,148.0,-0.23,1.73,13.63,XY24,NTT,FINANCE
1,ABB,7934.00,-1.46,-1.03,47.09,45.57,121922.0,-2707.0,258912.0,-36.40,64.0,H-MC,6.90,121.0,-0.02,1.92,14.97,AR,NTT,ELECTRICAL
33,HCLTECH,1908.19,-2.47,8.96,13.65,23.83,35982.0,21679.0,263603.0,13.61,52.0,X-LC,7.31,13.0,0.60,1.95,26.47,X40,ATH,IT
68,STARHEALTH,761.00,4.99,-8.47,60.89,47.26,156677.0,-23811.0,257312.0,24.79,65.0,H-SC,3.31,174.0,-0.15,1.91,37.86,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.82
1,25,44.88
2,50,76.80


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.02
MC    30.59
LC    24.37
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.47
X40      24.01
X40N     14.90
AR        8.87
XR        8.67
XY25      8.65
OX40N     7.49
SR        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.97
H-SC    22.97
X-LC    20.85
M-SC    11.13
X-SC    10.18
H-MC     4.80
M-MC     1.45
M-LC     1.25
L-LC     1.14
H-LC     1.13
L-SC     0.74
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.29,-14.81,53.95
FINANCE,13.00,-18.94,65.77
IT,12.98,-22.11,83.11
MISC,6.89,-39.57,96.66
ELECTRICAL,5.97,-16.46,58.95
PAINTS,5.46,-25.26,43.87
INSURANCE,4.87,-3.29,38.64
PHARMA,4.04,-7.78,41.71
AUTO,2.95,-31.64,77.17


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3399510.0,21
AR,1374107.0,10
XR,1319152.0,12
X40,1265791.0,15
X40N,1112997.0,10
OX40N,780453.0,10
XY25,453061.0,6
SR,295785.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3691781.0,24
X-MC,1797257.0,16
M-SC,1556539.0,15
X-LC,1076693.0,12
X-SC,924450.0,9
H-MC,431305.0,3
L-SC,190784.0,2
M-LC,123842.0,1
H-LC,80960.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1257429.0      6
           AR         948621.0      5
M-SC       XY24       882684.0      6
H-SC       XR         853680.0      7
X-MC       X40        613791.0      7
           XY24       489504.0      3
X-LC       X40        472393.0      6
X-MC       X40N       447782.0      4
X-SC       X40N       429520.0      4
M-SC       OX40N      349713.0      5
H-SC       OX40N      336266.0      4
X-SC       XY24       315323.0      3
H-SC       SR         295785.0      2
X-MC       XY25       246180.0      2
X-LC       XY24       236970.0      2
           X40N       235695.0      2
H-MC       XY24       217600.0      1
           AR         213705.0      2
M-SC       XR         193321.0      2
X-SC       X40        179607.0      2
X-LC       XY25       131635.0      2
M-SC       AR         130821.0      2
M-LC       XR         123842.0      1
L-SC       XR          96310.0      1
           OX40N       94474.0      1
H-LC       AR          80960.0      1
L-MC       XR          51999.0      1
M-MC       XY25        49563.0      1
L-LC       XY25        25683.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
